In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import layers

### Layers 

In [2]:
def FCLayer(size_inp, inp, nodes):
    W = tf.random_normal(shape=(size_inp,nodes), mean=0.0, stddev=1.0, dtype=tf.float32)
    #    W = tf.random_normal(shape=(tf.shape(inp)[1],nodes), mean=0.0, stddev=1.0, dtype=tf.float32)
    W = tf.Variable(W)
#    bias = tf.random_normal(shape=(1,nodes), mean=0.0, stddev=1.0, dtype=tf.float32)
#    bias = tf.Variable(bias)
    out = tf.matmul(inp,W)
    
    return W, out    

In [3]:
def reLULayer(inp):
    out = tf.maximum(0.0,inp)        
    return out

In [4]:
def Layer(size_inp, inp, nodes):
    W, outFC = FCLayer(size_inp, inp, nodes)
    outReLU = reLULayer(outFC)
    return W, outReLU

In [5]:
def network(size_inp, inp):
    W1, out1 = Layer(size_inp, inp, 4)
    
    W2, out2 = Layer(4, tf.nn.sigmoid(out1), 3)
    
    W3, out3 = Layer(3, tf.nn.sigmoid(out2), 2)
    
    return W1, W2, W3, out3
    

In [6]:
def inference(size_X, X):
#     return network(size_X, X)
   return tf.nn.sigmoid(network(size_X, X))


In [7]:
def evaluate(Yhat):
    predicted = tf.argmax(Yhat, dimension=1)
    return predicted
    #print output.eval(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32)))

In [8]:
def loss(Yhat, Ytrue):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Yhat, Ytrue))


In [9]:
def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

### Building the Network

In [10]:
# Parameters
learning_rate = 0.1
training_epochs = 1000
batch_size = 100
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 7 # MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.int32, [None, n_classes])

In [11]:
# Create model
def multilayer_perceptron(x):
    fc1 = layers.fully_connected(x, 256, activation_fn=tf.nn.relu)
    fc2 = layers.fully_connected(fc1, 256, activation_fn=tf.nn.relu)
    out = layers.fully_connected(fc2, n_classes, activation_fn=tf.nn.softmax)
    return out

In [12]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x)
print(pred.get_shape())
# Define loss and optimizer
# softmax = tf.nn.sparse_softmax_cross_entropy_with_logits(pred, y)

softmax = tf.nn.softmax_cross_entropy_with_logits(pred, y)
cost = tf.reduce_mean(softmax)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

(?, 2)


### Data

In [13]:
## Train Sequence
train_X = np.random.randint(0, 2, size=(20000, n_input))
train_Y = np.remainder(np.sum(train_X, axis=1),2)

# new_X = np.ones((train_X.shape[0], train_X.shape[1]+1))
# new_X[:, 1:] = train_X
# train_X = new_X

# Convert into one-hot vectors
num_labels = len(np.unique(train_Y))
new_Y = np.eye(num_labels)[train_Y]  # One liner trick!
train_Y = new_Y

X_size = train_X.shape[1] 

In [14]:
## Test Sequence
test_X = np.random.randint(0, 2, size=(10000, n_input))
test_Y = np.remainder(np.sum(test_X, axis=1),2)

# new_X = np.ones((test_X.shape[0], test_X.shape[1]+1))
# new_X[:, 1:] = test_X
# test_X = new_X

# Convert into one-hot vectors
num_labels = len(np.unique(test_Y))
new_Y = np.eye(num_labels)[test_Y]  # One liner trick!
test_Y = new_Y

### Train

In [15]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        # Loop over all batches
        # Run optimization op (backprop) and cost op (to get loss value)
        _, c, s = sess.run([optimizer, cost, softmax], feed_dict={x: train_X, y: train_Y})
        # Compute average loss
        avg_cost += c
        # Display logs per epoch step
        if epoch % display_step == 0:            
            correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost) + ", Train Accuracy:", accuracy.eval({x: train_X, y: train_Y}))
#             print(sess.run(pred, feed_dict={x: test_X, y: test_Y}), test_Y)
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Test Accuracy:", accuracy.eval({x: test_X, y: test_Y}))

Epoch: 0001 cost= 0.693084955, Train Accuracy: 0.51955
Epoch: 0101 cost= 0.688428044, Train Accuracy: 0.54965
Epoch: 0201 cost= 0.685062289, Train Accuracy: 0.61915
Epoch: 0301 cost= 0.680728257, Train Accuracy: 0.69225
Epoch: 0401 cost= 0.675278544, Train Accuracy: 0.76285
Epoch: 0501 cost= 0.667772770, Train Accuracy: 0.8029
Epoch: 0601 cost= 0.656104565, Train Accuracy: 0.86285
Epoch: 0701 cost= 0.637890637, Train Accuracy: 0.8735
Epoch: 0801 cost= 0.610171139, Train Accuracy: 0.9155
Epoch: 0901 cost= 0.566733122, Train Accuracy: 0.9546
Optimization Finished!
Test Accuracy: 0.9619
